In [1]:
!nvidia-smi

Sun Aug 27 05:54:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install -U -qq gdown kaggle

# Install Timm (Need to restart the runtime after finish install )
# !pip -qq install git+https://github.com/rwightman/pytorch-image-models.git
# !pip -qq install lightning transformers datasets evaluate pillow==9.2.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/all_AI_code/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !kaggle competitions download -c fruit-ripening

In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split, Dataset

from lightning.fabric import Fabric

import evaluate

# Pytorch Image model (TIMM) library: a library for state-of-the-art image classification
import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold,KFold

from PIL import Image
import cv2


import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from glob import glob
from pathlib import Path
from copy import copy
import time
import shutil
import random
import zipfile

In [4]:
# with zipfile.ZipFile("./fruit-ripening.zip", "r") as zip_ref:
#     print("Unzipping main_zip data...")
#     zip_ref.extractall('/content/data/')

# os.remove("./fruit-ripening.zip")

In [5]:
df_submit = pd.read_csv('/content/data/submission.csv')
df_submit['path'] = '/content/data/test/test/' + df_submit['FileName']
df_submit.head()

,FileName,Class,path
0,abdy9380.png,0.0,/content/data/test/test/abdy9380.png
1,adua0170.png,0.0,/content/data/test/test/adua0170.png
2,aebq0222.png,5.0,/content/data/test/test/aebq0222.png
3,aeew6117.png,NaN,/content/data/test/test/aeew6117.png
4,aegl6733.png,NaN,/content/data/test/test/aegl6733.png


In [6]:
ls = glob('/content/data/train/train/*/*.png')
df_data = pd.DataFrame(ls,columns = ['path'])
df_data['label'] = df_data['path'].map(lambda x:int(x.split('/')[-2]))
df_data.sample(5)

,path,label
931,/content/data/train/train/0/IMG_20220426_20544...,0
747,/content/data/train/train/0/IMG_20220427_16400...,0
2634,/content/data/train/train/4/IMG_20220427_16192...,4
1756,/content/data/train/train/0/IMG_20220426_17153...,0
2022,/content/data/train/train/0/IMG_20220427_15335...,0


In [7]:
df_data['label'].value_counts()

0    2061
1     202
2     127
3      86
6      59
4      59
5      53
7      41
Name: label, dtype: int64

In [8]:
# Transform image data based on ImageNet's mean and std
mean = torch.tensor([0.5, 0.5, 0.5])
std = torch.tensor([0.5, 0.5, 0.5])
data_transforms = {
    "train": T.Compose([
        T.CenterCrop(450),
        T.RandomHorizontalFlip(p=0.5),
        T.RandomVerticalFlip(p=0.5),
        T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        T.RandomRotation(degrees=(-15, 15)),
        T.RandomAutocontrast(),
        T.Resize((384, 384)),
        T.ToTensor(),
        T.Normalize(mean=mean, std=std)
    ]),
    "valid": T.Compose([
        T.CenterCrop(450),
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(degrees=(-10, 10)),
        T.Resize((384, 384)),
        T.ToTensor(),
        T.Normalize(mean=mean, std=std)
    ]),
    "test": T.Compose([
        T.CenterCrop(450),
        T.Resize((384, 384)),
        T.ToTensor(),
        T.Normalize(mean=mean, std=std)
    ])
}

In [9]:
class trainDataset(Dataset):
    def __init__(self, df = df_data, transforms=data_transforms, mode='train', sample=600, fullData=False):
      if fullData:
        self.data = df_data.sample(frac=1).reset_index(drop=True)
        self.transforms = transforms[mode]
      else:
        self.data_1 = df[df['label']==0].sample(sample)
        self.data_2 = df[df['label']!=0].sample(sample)
        self.data = pd.concat([self.data_1,self.data_2]).sample(frac=1).reset_index(drop=True)
        self.transforms = transforms[mode]
    def __len__(self):
        return len(self.data)

    def preprocess(self, img_path):
      img_rgb = cv2.imread(img_path)
      img = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
      mask = np.zeros(img.shape, dtype=np.uint8)
      _,th = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
      dl_img = cv2.dilate(th, kernel=np.ones((5, 5), np.uint8), iterations=1)
      dl_img[:,:150] = 0
      dl_img[:,650:] = 0
      dl_img = cv2.erode(dl_img, kernel=np.ones((3, 3), np.uint8), iterations=1)
      contours, hierarchy = cv2.findContours(dl_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      max_c = max(contours, key = cv2.contourArea)
      mask = cv2.fillPoly(mask, pts=[max_c], color=255)
      final = cv2.bitwise_and(img_rgb, img_rgb, mask = mask)
      destRGB = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
      return destRGB

    def __getitem__(self, index):
        path = self.data['path'][index]
        label = int(path.split('/')[-2])
        img = self.preprocess(path)
        img = Image.fromarray(img)
        img = self.transforms(img)
        return img, label

In [10]:
train_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'train', sample = 600, fullData = False)
print("total images :",len(train_dataset))
# fig,axe = plt.subplots(1,7,figsize=(18, 8))
# for i in range(7):
#     img = train_dataset[i][0]
#     axe[i].imshow(np.array(img))
#     axe[i].axis('off')
# plt.show()

total images : 1200


In [11]:
torch.set_float32_matmul_precision('high')
fabric = Fabric(accelerator="cuda") # , precision="16-mixed"
fabric.launch()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
# !pip -q install GPUtil

In [13]:
import GPUtil

def free_gpu_cache():
    print("Initial GPU Usage")
    GPUtil.showUtilization()
    torch.cuda.empty_cache()
    print("GPU Usage after emptying the cache")
    GPUtil.showUtilization()

In [14]:
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [15]:
timm.list_models('*maxvit_base*', pretrained=True)

['maxvit_base_tf_224.in1k',
 'maxvit_base_tf_224.in21k',
 'maxvit_base_tf_384.in1k',
 'maxvit_base_tf_384.in21k_ft_in1k',
 'maxvit_base_tf_512.in1k',
 'maxvit_base_tf_512.in21k_ft_in1k']

In [16]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
all_eval_scores = []

# Cross Validation Configuration
metric = evaluate.load("f1")
train_batch_size = 4
eval_batch_size = 4
num_accumulate = 8
num_epochs = 30

# Load Model
model_name = "maxvit_base_tf_384.in21k_ft_in1k"
model = timm.create_model(model_name, pretrained=True, num_classes=8)

# Load Optimizer and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = timm.optim.create_optimizer_v2(model, opt="AdamW", lr=5e-4)
# optimizer = timm.optim.Lookahead(optimizer, alpha=0.5, k=6)    # update the slow weight every k steps
                                                                # update the optimizer by combine slow weight and fast weight * alpha

model, optimizer = fabric.setup(model, optimizer)

scheduler = timm.scheduler.create_scheduler_v2(optimizer, num_epochs=num_epochs)[0]

info = {
        "metric_train": [],
        "metric_val": [],
        "train_loss": [],
        "val_loss": [],
        "best_metric_val": -999,
        "best_val_loss": 999,
    }
for epoch in range(num_epochs):
    train_loss_epoch = []
    val_loss_epoch = []

    train_preds = []
    train_targets = []

    val_preds = []
    val_targets = []

    if epoch%5==0:
      train_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'train', sample = 600, fullData = True)
    else:
      train_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'train', sample = 600, fullData = False)

    valid_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'valid', sample = 400)

    train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_dataloader = DataLoader(valid_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)


    train_dataloader = fabric.setup_dataloaders(train_dataloader)
    val_dataloader = fabric.setup_dataloaders(val_dataloader)
    num_updates = epoch * len(train_dataloader)

    ### === Train Loop === ###
    ## Time
    start_time = time.monotonic()

    model.train()
    for idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch: {epoch+1} | Training ", leave=False)):
        inputs, targets = batch
        outputs = model(inputs.to(device))
        loss = criterion(outputs, targets.to(device))

        fabric.backward(loss)

        # === Gradient Accumulation === #
        if ((idx + 1) % num_accumulate == 0) or (idx + 1 == len(train_dataloader)):
            optimizer.step()
            scheduler.step_update(num_updates=num_updates)
            optimizer.zero_grad()
        # ============================= #

        train_loss_epoch.append(loss.item())
        train_preds += outputs.argmax(-1).detach().cpu().tolist()
        train_targets += targets.tolist()
    ### ==================== ###

    # optimizer.sync_lookahead()              # Sync slow weight and fast weight
    scheduler.step(epoch + 1)

    ### === Evaluation Loop === ###
    if epoch%3==0 or epoch>9:
      model.eval()
      with torch.no_grad():
          for batch in tqdm(val_dataloader, desc=f"Epoch: {epoch+1} | Evaluating ", leave=False):
              inputs, targets = batch
              outputs = model(inputs.to(device))
              loss = criterion(outputs, targets.to(device))

              # Log Values
              val_loss_epoch.append(loss.item())
              val_preds += outputs.argmax(-1).detach().cpu().tolist()
              val_targets += targets.tolist()

      metric_val = metric.compute(predictions=val_preds, references=val_targets, average='macro')['f1']
      info["metric_val"].append(metric_val)
      info["val_loss"].append(np.average(val_loss_epoch))

    else:
      metric_val = 0.0
      val_loss_epoch = 0.0

    ### ======================= ###
    ## Time train finish
    end_time = time.monotonic()

    # Log Data
    metric_train = metric.compute(predictions=train_preds, references=train_targets, average='macro')['f1']
    info["metric_train"].append(metric_train)
    info["train_loss"].append(np.average(train_loss_epoch))


    if metric_val > info["best_metric_val"]:
    # if info["val_loss"][-1] < info["best_val_loss"]:
        # print("New Best Score!")
        # print("New Best Val Loss")
        info["best_metric_val"] = metric_val
        # info["best_val_loss"] = info["val_loss"][-1]
        torch.save(model, f"/content/maxvit_base_tf_384_best.pt")

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    str_time = f"{epoch_mins}m {epoch_secs}s"
    print(f"Epoch: {epoch+1} : {str_time.ljust(7)} | Loss : train {np.average(train_loss_epoch):.4f} valid {np.average(val_loss_epoch):.4f} | f1-macro : train {metric_train*100:.2f}% valid {metric_val*100:.2f}%")
    # print(info)
    # save all best metric val
    # all_eval_scores.append(info["best_metric_val"])

Epoch: 1 | Training :   0%|          | 0/672 [00:00<?, ?it/s]

Epoch: 1 | Evaluating :   0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 1 : 11m 55s | Loss : train 0.6062 valid 0.7794 | F1 : train 27.55% valid 42.26%


Epoch: 2 | Training :   0%|          | 0/300 [00:00<?, ?it/s]

Epoch: 2 : 4m 47s  | Loss : train 0.7778 valid 0.0000 | F1 : train 43.40% valid 0.00%


...

Epoch: 29 | Evaluating :   0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 29 : 5m 54s  | Loss : train 0.0072 valid 0.0094 | F1 : train 99.77% valid 99.75%


Epoch: 30 | Training :   0%|          | 0/300 [00:00<?, ?it/s]

Epoch: 30 | Evaluating :   0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 30 : 5m 53s  | Loss : train 0.0051 valid 0.0152 | F1 : train 100.00% valid 99.62%


In [20]:
# save latest trained model at epoch 30
torch.save(model, f"/content/maxvit_base_tf_384_ep30.pt")

In [21]:
# prediction report base on balance classes at model ep30
fake_test_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'test', sample = 600)
fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)
loaded_model = torch.load(f"/content/maxvit_base_tf_384_ep30.pt")
ls_ans_fake_test = []
ls_target =[]

loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(fake_test_dataloader, desc="Evaluating ", leave=False):
        inputs, targets = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans_fake_test.extend(y_pred.tolist())
        ls_target.extend(targets.tolist())
print(classification_report(ls_ans_fake_test, ls_target))

Evaluating :   0%|          | 0/300 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       599
           1       1.00      0.99      1.00       194
           2       0.99      1.00      1.00       117
           3       1.00      0.99      0.99        87
           4       1.00      1.00      1.00        57
           5       1.00      1.00      1.00        49
           6       1.00      1.00      1.00        58
           7       1.00      1.00      1.00        39

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200



In [19]:
# prediction report base on balance classes at best model
fake_test_dataset = trainDataset(df_data,transforms = data_transforms, mode = 'test', sample = 600)
fake_test_dataloader = DataLoader(fake_test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)
loaded_model = torch.load(f"/content/maxvit_base_tf_384_best.pt")
ls_ans_fake_test = []
ls_target =[]

loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(fake_test_dataloader, desc="Evaluating ", leave=False):
        inputs, targets = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans_fake_test.extend(y_pred.tolist())
        ls_target.extend(targets.tolist())
print(classification_report(ls_ans_fake_test, ls_target))

Evaluating :   0%|          | 0/300 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       599
           1       0.99      1.00      0.99       189
           2       0.99      0.98      0.98       121
           3       1.00      0.99      0.99        86
           4       1.00      1.00      1.00        56
           5       1.00      1.00      1.00        53
           6       1.00      1.00      1.00        58
           7       1.00      1.00      1.00        38

    accuracy                           1.00      1200
   macro avg       1.00      1.00      1.00      1200
weighted avg       1.00      1.00      1.00      1200



In [18]:
class testDataset(Dataset):
    def __init__(self, df = df_data, transforms = data_transforms, mode = 'test'):
        self.data = df
        self.transforms = transforms[mode]

    def __len__(self):
        return len(self.data)

    def preprocess(self, img_path):
      img_rgb = cv2.imread(img_path)
      img = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
      mask = np.zeros(img.shape, dtype=np.uint8)
      _,th = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
      dl_img = cv2.dilate(th, kernel=np.ones((5, 5), np.uint8), iterations=1)
      dl_img[:,:150] = 0
      dl_img[:,650:] = 0
      dl_img = cv2.erode(dl_img, kernel=np.ones((3, 3), np.uint8), iterations=1)
      contours, hierarchy = cv2.findContours(dl_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      max_c = max(contours, key = cv2.contourArea)
      mask = cv2.fillPoly(mask, pts=[max_c], color=255)
      final = cv2.bitwise_and(img_rgb, img_rgb, mask = mask)
      destRGB = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
      return destRGB[100:500,200:600]

    def __getitem__(self, index):
        path = self.data['path'][index]
        img = self.preprocess(path)
        img = Image.fromarray(img)
        img = self.transforms(img)
        return img

test_dataset = testDataset(df_submit, transforms=data_transforms, mode='test')
test_dataloader = DataLoader(test_dataset, batch_size=eval_batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [29]:
ls_ans = []

loaded_model = torch.load(f"/content/maxvit_base_tf_384_best.pt")
loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating ", leave=False):
        inputs = batch
        outputs = loaded_model(inputs.to(device))
        y_pred = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
        ls_ans.extend(y_pred.tolist())

Evaluating :   0%|          | 0/169 [00:00<?, ?it/s]

In [30]:
ls_ans[:5]

[0, 0, 5, 0, 0]

In [31]:
df_submit['Class'][3:] = ls_ans[3:]
df_submit['Class'] = df_submit['Class'].map(int)
df_submit

<ipython-input-31-3d8e79339bbb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_submit['Class'][3:] = ls_ans[3:]


,FileName,Class,path
0,abdy9380.png,0,/content/data/test/test/abdy9380.png
1,adua0170.png,0,/content/data/test/test/adua0170.png
2,aebq0222.png,5,/content/data/test/test/aebq0222.png
3,aeew6117.png,0,/content/data/test/test/aeew6117.png
4,aegl6733.png,0,/content/data/test/test/aegl6733.png
...,...,...,...
668,zvxc1988.png,5,/content/data/test/test/zvxc1988.png
669,zyms7894.png,0,/content/data/test/test/zyms7894.png
670,zzcb3748.png,0,/content/data/test/test/zzcb3748.png
671,zzts1908.png,1,/content/data/test/test/zzts1908.png


In [25]:
# maxvit_base_tf_384_ep30
df_submit['Class'].value_counts()

0    526
1     47
2     33
5     21
3     14
7     12
6     11
4      9
Name: Class, dtype: int64

In [32]:
# maxvit_base_tf_384_best
df_submit['Class'].value_counts()

0    524
1     45
2     36
5     21
3     15
7     12
6     11
4      9
Name: Class, dtype: int64

In [33]:
df_submit[['FileName','Class']].to_csv('maxvit_base_tf_384_best.csv', index=False)

In [34]:
!kaggle competitions submit -c fruit-ripening -f maxvit_base_tf_384_best.csv -m "maxvit_base_tf_384_best"

100% 9.87k/9.87k [00:02<00:00, 3.81kB/s]
Successfully submitted to Hackathon Online: Image Processing